In [ ]:
import pygame
import torch
import time
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from NovelSwarmBehavior.novel_swarms.config.EvolutionaryConfig import GeneticEvolutionConfig
from NovelSwarmBehavior.novel_swarms.config.WorldConfig import RectangularWorldConfig
from NovelSwarmBehavior.novel_swarms.config.AgentConfig import DiffDriveAgentConfig
from NovelSwarmBehavior.novel_swarms.sensors.SensorSet import SensorSet
from NovelSwarmBehavior.novel_swarms.sensors.BinaryLOSSensor import BinaryLOSSensor
from NovelSwarmBehavior.novel_swarms.sensors.GenomeDependentSensor import GenomeBinarySensor
from NovelSwarmBehavior.novel_swarms.novelty.GeneRule import GeneBuilder
from NovelSwarmBehavior.novel_swarms.config.defaults import ConfigurationDefaults
from NovelSwarmBehavior.novel_swarms.novelty.GeneRule import GeneRule
from NovelSwarmBehavior.novel_swarms.config.OutputTensorConfig import OutputTensorConfig
from generation.evolution import ModifiedNoveltyArchieve
from PIL import Image
from sklearn_extra.cluster import KMedoids
import cv2
import numpy as np
import torch
from data.swarmset import ContinuingDataset, SwarmDataset
from networks.embedding import NoveltyEmbedding
from generation.evolution import ModifiedHaltingEvolution
from networks.archive import DataAggregationArchive
from hil.HIL import HIL
import time
from networks.ensemble import Ensemble

dataset = ContinuingDataset("/home/connor/Desktop/Experiments/GECCO_2023/Brown-two-sensor-no-h/Experiment_1/1675638750-brown-two-sensor-no-h", folder_name="data", create=False)
network = None
medoids = 12
cap_data = 10000

print(dataset[0][1])

def resizeInput(X, w=200):
    frame = X.astype(np.uint8)
    resized = cv2.resize(frame, dsize=(w, w), interpolation=cv2.INTER_AREA)
    return resized

archive = ModifiedNoveltyArchieve()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for i in range(len(dataset)):
    anchor_encoding, genome, behavior = dataset[i][0], dataset[i][1], dataset[i][2]
    if network is not None:
        anchor_encoding = resizeInput(anchor_encoding, (50, 50))
        anchor_encoding = torch.from_numpy(anchor_encoding).to(device).float()
        network.eval()
        embedding = network(anchor_encoding.unsqueeze(0)).squeeze(0).cpu().detach().numpy()
        archive.addToArchive(vec=embedding, genome=genome)
    else:
        archive.addToArchive(vec=behavior, genome=genome)

kmedoids = KMedoids(n_clusters=medoids, random_state=0).fit(archive.archive)
medoids = kmedoids.medoid_indices_
labels = kmedoids.labels_

import matplotlib.pyplot as plt
for i, medoid_i in enumerate(medoids):
    medoid_image = dataset[medoid_i][0]
    plt.imshow(medoid_image, cmap="Greys")

[-0.3 -0.9 -0.4  0.3  1.  -0.7  0.4 -0.3 -0.1]
